In [105]:
from backtesting import Backtest
from backtesting.test import SMA, GOOG
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import wandb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [106]:
ticket_symbol = "DAX"

ticker = yf.Ticker(ticket_symbol)

historical_data = ticker.history(period="2y")  # data for last year

In [107]:
LAST_DAYS = -90
predict_Days = historical_data[LAST_DAYS:]

train_data = historical_data[:LAST_DAYS]
columns_to_drop = ["Close"]

X_train = train_data.drop(columns=columns_to_drop)
Y_train = train_data["Close"].shift(-1)

X_test = predict_Days.drop(columns=columns_to_drop)
Y_test = predict_Days["Close"].shift(-1)

X_scaler = StandardScaler()
X_train_norm = pd.DataFrame(X_scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_norm = pd.DataFrame(X_scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

Y_scaler = StandardScaler()
Y_train_norm = pd.Series(
    Y_scaler.fit_transform(Y_train.dropna().values.reshape(-1, 1)).flatten(), index=Y_train.dropna().index, name="Close"
)
Y_test_values = Y_test.dropna().values.reshape(-1, 1)
Y_test_norm = pd.Series(Y_scaler.transform(Y_test_values).flatten(), index=Y_test.dropna().index, name="Close")


model_norm = ARIMA(
    Y_train_norm.dropna(),
    order=(5, 0, 5),
    seasonal_order=(2, 0, 0, 12),
)
model_fit_norm = model_norm.fit()

forecast_norm = model_fit_norm.forecast(steps=len(predict_Days))
forecast_orig = Y_scaler.inverse_transform(forecast_norm.values.reshape(-1, 1)).flatten()
forecast_series = pd.Series(forecast_orig, index=predict_Days.index)

/Users/filippatyk/miniforge3/envs/uam/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/filippatyk/miniforge3/envs/uam/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/filippatyk/miniforge3/envs/uam/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/filippatyk/miniforge3/envs/uam/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge.

In [108]:
class ARIMAStrategy(Strategy):
    """
    buy if the forecast is higher than the previous two forecasts
    sell if the forecast is lower than the previous two forecasts
    """

    buy_sl = 0.95  # stop-loss multiplier for long positions
    buy_tp = 1.05  # take-profit multiplier for long positions
    sell_sl = 1.01  # stop-loss multiplier for short positions
    sell_tp = 0.99  # take-profit multiplier for short positions

    def init(self):
        pass

    def next(self):
        t = self.data.index[-1]
        if t not in forecast_series.index:
            return

        pos = forecast_series.index.get_loc(t)
        if pos < 2:
            return
        current_pred = forecast_series.loc[t]
        forecast_prev = forecast_series.iloc[pos - 1]
        forecast_prev_prev = forecast_series.iloc[pos - 2]

        if current_pred > forecast_prev and forecast_prev > forecast_prev_prev:
            if not self.position:
                self.sell(sl=self.data.Close[-1] * self.sell_sl, tp=self.data.Close[-1] * self.sell_tp)
        elif current_pred < forecast_prev and forecast_prev < forecast_prev_prev:
            if not self.position:
                self.buy(sl=self.data.Close[-1] * self.buy_sl, tp=self.data.Close[-1] * self.buy_tp)

In [109]:
bt = Backtest(predict_Days, ARIMAStrategy, cash=10_000, commission=0.002, exclusive_orders=True)

stats = bt.run()
print(stats)
bt.plot()

Backtest.run:   0%|          | 0/89 [00:00<?, ?bar/s]

Start                     2024-12-04 00:00...
End                       2025-04-15 00:00...
Duration                    131 days 23:00:00
Exposure Time [%]                    92.22222
Equity Final [$]                  10850.29626
Equity Peak [$]                   11115.97996
Commissions [$]                     323.88111
Return [%]                            8.50296
Buy & Hold Return [%]                12.15402
Return (Ann.) [%]                     25.6714
Volatility (Ann.) [%]                30.71722
CAGR [%]                              16.8645
Sharpe Ratio                          0.83573
Sortino Ratio                         1.78732
Calmar Ratio                          1.92445
Alpha [%]                            -1.55972
Beta                                  0.82793
Max. Drawdown [%]                   -13.33958
Avg. Drawdown [%]                    -3.06061
Max. Drawdown Duration       48 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                          

GridPlot(id='p12819', ...)